# Advanced Prompt Engineering 
___

This notebook will take the prompt engineering techniques that were taught earlier, and add LangChain to the mix. LangChain is to the LLMs what 'Education' is to Humans. It offers unique ways to add to talk and interact with the LLMs.

In this notebook we will introduce simple LangChain codes that can be taken as is, and can be used to customize the responses from ChatGPT. You will also see how the LLMs (or GPT-3.5) actually 'thinks' before it delivers its responses.

At the end of the module, we will also provide a few reading resources that you can use and add to this notebook. For this notebook we will use 'GPT-3.5-turbo' due to its affordability and easy generalizability.

Finally we will build an interactive pizza delivery app that will help you understand how we can integrate the API into different applications!


**This notebook utilizes a paid API key from ChatGPT, hence, do not re-run the cells'**

___




In [20]:
#libraries
from openai import OpenAI
import openai
import os 

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

___
## Section 1 : Components and Basic Tasks
___

The basic code for connecting and getting responses for ChatGPT:

1. **Model** : Defining the model of choice

2. **Role** : Taking the concept of 'actors' a bit further, the role differentiates between :
  - System : To programmatically define the model's attitude 
  - User : To define user questions
  - Content : An example response to aid the understanding of the expected response from the model

3. **Temperature** : Selects the degree of randomness/creativity of the model's responses in the range of 0-1.

  - For instance, setting the temperature to 0.1 prompts a more focused and expected answer, while 0.8 encourages a more creative response. The temperature settings range from 0 to 1, and the right balance depends on the user’s needs.



In [3]:
# helper function to get the prompt and completion:

client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

As with the initial notebook, it is important to structure the model's responses in a certain way using certain techniques.

#### Using delimiters:

- Use delimiters to clearly indicate distinct parts of the input
  - Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`
- Ask for structured outputs
- Check whether conditions are satisfied, check the assumptions for the task

- Few shot prompting : adding success cases

Now let's get on the basic tasks that the user can do.



### 1. Text Summarization:

In [4]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
display(HTML(response))

### 2. Getting Structured outputs:

In [5]:
# JSON outputs
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Whispers in the Wind",
        "author": "Lucas Blackwood",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Echoes of the Past",
        "author": "Samantha Greene",
        "genre": "Historical Fiction"
    }
]


In [6]:
# Getting step by step instructions
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


### 3. Altering model behavior:

In [7]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The tallest trees withstand the strongest storms; the brightest stars shine through the darkest nights; the strongest hearts endure the toughest trials.


In [8]:
# Prompt
prompt_1 = """
Talk to me about the solar system as Mary.
"""

# Messages 
messages = [
    {'role': 'system', 'content': 'Mary is a factual chatbot that is also sarcastic.'},
    {'role': 'user', 'content': "What's the capital of France?"},
    {'role': 'assistant', 'content': "Paris, as if everyone doesn't already know!"},
    {'role': 'user', 'content':f"{prompt_1}"}
]

# Function to add new messages and set temperature
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    messages_formatted = [{"role": msg["role"], "content": msg["content"]} for msg in messages]
    response = client.chat.completions.create(
        model=model,
        messages=messages_formatted,
        temperature=temperature
    )
    return response.choices[0].message.content

# Generate a response using the adjusted function and messages
response = get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Oh, the solar system, where the sun is the center of attention and all the planets revolve around it like they're in some cosmic dance. You've got Mercury, Venus, Earth (the best one, obviously), Mars, Jupiter, Saturn, Uranus, and Neptune, all spinning around in space like a well-choreographed ballet. And then there's Pluto, the little rebel that got kicked out of the planet club. Poor Pluto, always causing controversy.


In [9]:
# adjusting the temperature for more sarcasm

response = get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Alright, strap in because we're about to take a quick tour of our cosmic neighborhood. So, in the center of the solar system, we have the Sun, which is like the big boss keeping all the planets in check. Then we have the inner rocky planets - Mercury, Venus, Earth, and Mars - which are all quite high-maintenance if you ask me. Further out, you've got the gas giants - Jupiter, Saturn, Uranus, and Neptune - who are basically the cool kids of the solar system. And let's not forget about all the dwarf planets, asteroids, comets, and other debris floating around causing cosmic chaos. It's a wild ride out there, folks.


### 4. Language Translation:
- It is important for language translation tasks to give the model time to think.

In [10]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune along the way.

2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.",
  "num_names": 2
}


___
## Section 2. Improving Model's responses for simple tasks
___

### Giving the model time to think:
- Specify the steps required to complete the task
- Instruct the model to run its own solution before running to its own conclusion

### Limitations:
- Model Hallucination : When the model doesn't know it's own boundary


In [11]:
# Wrong Prompt:

prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

# example of the wrong solution:

response = get_completion(prompt)
print("Wrong solution:")
print(response)

Wrong solution:
The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


In [12]:
# correct prompt:

# Correct prompt :
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

response = get_completion(prompt)
print("Correct solution:")
print(response)

Correct solution:
Let's calculate the total cost for the first year of operations as a function of the number of square feet.

Given:
- Land costs $100 / square foot
- Solar panels cost $250 / square foot
- Maintenance contract: $100,000 flat + $10 / square foot

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100x
2. Solar panel cost: $250x
3. Maintenance cost: $100,000 + $10x

Total cost: $100x + $250x + $100,000 + $10x = $360x + $100,000

Therefore, the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000. 

Is the student's solution the same as the actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


Play around with the prompting techniques that were taught earlier and optimize the prompts for your benefit!

___
## Section 3. Creating an OrderBot in Python GUI
___

### OrderBot : Helping the users order pizza
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 


In [13]:
import panel as pn

# Initialize the Panel extension
pn.extension()

# Helper function to interact with OpenAI API
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

# Function to handle the conversation logic
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


# UI components

inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
# button_conversation.on_click(collect_messages)  # Attach event handler

interactive_conversation = pn.bind(collect_messages, button_conversation)


display_area = pn.Column()  # Dynamic area to display conversation

# Build the dashboard
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
    display_area
)

# Serve or display the dashboard
dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'8e33147a-8f00-4cf8-a1a8-004485642370': {'version…

<p align="center">
  <img src="chat.png" />
</p>

In [14]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 
response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "order": {
        "pizza": {
            "type": "cheese",
            "size": "large"
        },
        "toppings": [
            "peppers"
        ],
        "drinks": [
            {
                "type": "coke",
                "size": "largest"
            }
        ],
        "sides": [
            {
                "item": "fries",
                "size": "large"
            }
        ],
        "total_price": 24.45
    }
}
```


Go through the Chat and see how the LLM handles responses! It understands the user, then acts according to the given role!



___
## Section 4. Simple Langchain Prompts
___

In the following section we will integrate a few components of langchain to see how the LLM's think. Don't worry if you can't understand the code behind it. 

This module is just for you to develop a basic intuition about how the model thinks!


In [21]:
from langchain_openai import ChatOpenAI

llm_model = 'gpt-3.5-turbo-0301'
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000152C9B63110>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000152C9B6E190>, model_name='gpt-3.5-turbo-0301', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

- LangChain has its own 'Prompt' templates to help the user add the 'role','content' and other functionalities
- It can create memory buffers to store the context of conversations and create even more complex models.

The code below will show how the Langchain helps us create and store templates so that we can continuously interact with the model in the same role.

In [22]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt


PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [23]:
# for instance:
customer_style = """American English \
in a calm and respectful tone
"""

# Imagine a Pirate as our customer:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages)


<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]


In [24]:
print('Model Response :')
print(chat(customer_messages))


Model Response :


content="I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add insult to injury, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"


#### Using LangChain agents to problems.
LangChain agents are a functional tool that can help the user train or add a specific functionality to get accurate reponses from the LLM. 

Given below are two such agents, Math and Python.

In [25]:
# libraries:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

import warnings
warnings.filterwarnings('ignore')

In [26]:
# Maths Problems:

llm = ChatOpenAI(temperature=0, model=llm_model)

# tools to use :
tools = load_tools(["llm-math"], llm=llm) # "wikipedia"

# initialize the agent :
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # chat : best tp work with chat type models, # REACT : optimised for reasoning
    handle_parsing_errors=True, # llm might output something that we might not be able to parse (ask it to correct itself)
    verbose = True) # To see what the LLM 'Thinks'

agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Question: What is the 25% of 300?
Thought: I need to use a calculator to find the answer.
Action:
```
{
  "action": "Calculator",
  "action_input": "0.25*300"
}
```

Observation: Answer: 75.0
Thought:The answer is 75.0
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [27]:
# Coding problems:

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(), # a way for llm to use python
    verbose=True
)

customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python_REPL
Action Input: 
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted(customers, key=lambda x: (x[1], x[0]))
```
Observation: 
Thought:I now know the final answer
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"

- The elements in Green represent how the LLM actually thinks!

In [28]:
# to get in depth information:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [1.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.\nAction: Python_REPL\nAction Input: \n```\ncustomers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\nsorted(customers, key=lambda x: (x[1], x[0]))\n```",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content

___

Treat this notebook essentially as a gateway to what LLMs can actually do! Create new ideas, fine-tune new models, change their behavior, just play around !

___